# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [373]:
problem = '''You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem. '''

## 2. Generate the mathematical model

In [443]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [444]:
#Show the resopnse in a formatted way
Markdown(response.text)

Let's think step by step to formulate a mathematical optimization model for the problem:

**Sets:**

*  $I$: Set of items, indexed by $i$.
*  $B$: Set of boxes, indexed by $j$.

**Parameters:**

*  $w_i$: Weight of item $i$, $\forall i \in I$.
*  $C$: Capacity of each box.
*  $N$: Maximum number of boxes allowed.

**Decision Variables:**

*  $x_{ij}$: Binary variable equal to 1 if item $i$ is packed in box $j$, 0 otherwise, $\forall i \in I, \forall j \in B$.

**Objective Function:**
Minimize the number of boxes used. 

$$\text{Minimize} \sum_{j \in B} \left( 1 - \prod_{i \in I} (1 - x_{ij}) \right) $$

This objective function utilizes a clever trick:

*   For each box $j$, if no item is packed in it (all $x_{ij} = 0$), the product term $\prod_{i \in I} (1 - x_{ij})$ evaluates to 1. Hence, $1 - \prod_{i \in I} (1 - x_{ij})$ becomes 0, indicating the box is unused.
*   If at least one item is packed in box $j$ (at least one $x_{ij} = 1$), the product term $\prod_{i \in I} (1 - x_{ij})$ evaluates to 0. Consequently, $1 - \prod_{i \in I} (1 - x_{ij})$ becomes 1, signifying the box is used.

**Constraints:**

1. **Capacity Constraint:**  The total weight of items packed in each box cannot exceed the box capacity.
    $$\sum_{i \in I} w_i x_{ij} \leq C, \forall j \in B$$

2. **One Box per Item:** Each item must be packed in exactly one box.
    $$\sum_{j \in B} x_{ij} = 1, \forall i \in I$$

3. **Box Limit Constraint:** The total number of boxes used cannot exceed the given limit.
   $$\sum_{j \in B} \left( 1 - \prod_{i \in I} (1 - x_{ij}) \right) \leq N $$

This mathematical optimization model is a mixed-integer linear programming (MILP) model, which can be solved using specialized optimization software.


## 3. Generate the pyomo code

In [445]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [446]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (You can replace this with your actual data)
items = ['A', 'B', 'C', 'D', 'E']
weights = {'A': 4, 'B': 2, 'C': 7, 'D': 5, 'E': 3}
capacity = 10
max_boxes = 5

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.B = pyo.Set(initialize=range(max_boxes))

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Variables
model.x = pyo.Var(model.I, model.B, domain=pyo.Binary)

# Objective function: Minimize the number of boxes used
def obj_rule(model):
    return sum(1 - pyo.prod(1 - model.x[i, j] for i in model.I) for j in model.B)
model.obj = pyo.Objective(rule=obj_rule)

# Constraints
# Capacity constraint
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C
model.capacity_constraint = pyo.Constraint(model.B, rule=capacity_rule)

# One box per item constraint
def one_box_rule(model, i):
    return sum(model.x[i, j] for j in model.B) == 1
model.one_box_constraint = pyo.Constraint(model.I, rule=one_box_rule)

# Box limit constraint
def box_limit_rule(model):
    return sum(1 - pyo.prod(1 - model.x[i, j] for i in model.I) for j in model.B) <= model.N
model.box_limit_constraint = pyo.Constraint(rule=box_limit_rule)

# Solve
solver = pyo.SolverFactory('glpk') # You can choose a different solver if you have it installed
solver.solve(model)

# Print results
print("Optimal Solution:")
for j in model.B:
    if sum(model.x[i, j].value for i in model.I) > 0: # Check if box j is used
        print(f"Box {j+1}:", [i for i in model.I if model.x[i, j].value == 1])

print("\nNumber of boxes used:", model.obj())
```

In this code:

*   `items` and `weights` are sample data. You would replace these with your actual sets and parameters.
*   The rest of the code constructs the Pyomo model based on the provided mathematical formulation.
*   The `glpk` solver is used as an example. You can choose a different solver if you have it installed.
*   The code then prints which items are packed in which boxes and the total number of boxes used in the optimal solution.


## 4. Input problem data and try running the generated code

In [451]:
import pyomo.environ as pyo

# Sample Data (You can replace this with your actual data)
items = range(1,25)
weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
capacity = 9
max_boxes = 13

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.B = pyo.Set(initialize=range(max_boxes))

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Variables
model.x = pyo.Var(model.I, model.B, domain=pyo.Binary)

# Objective function: Minimize the number of boxes used
def obj_rule(model):
    return sum(1 - pyo.prod(1 - model.x[i, j] for i in model.I) for j in model.B)
model.obj = pyo.Objective(rule=obj_rule)

# Constraints
# Capacity constraint
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C
model.capacity_constraint = pyo.Constraint(model.B, rule=capacity_rule)

# One box per item constraint
def one_box_rule(model, i):
    return sum(model.x[i, j] for j in model.B) == 1
model.one_box_constraint = pyo.Constraint(model.I, rule=one_box_rule)

# Box limit constraint
def box_limit_rule(model):
    return sum(1 - pyo.prod(1 - model.x[i, j] for i in model.I) for j in model.B) <= model.N
model.box_limit_constraint = pyo.Constraint(rule=box_limit_rule)

# Solve
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you have it installed
solver.solve(model)

# Print results
print("Optimal Solution:")
for j in model.B:
    if sum(model.x[i, j].value for i in model.I) > 0: # Check if box j is used
        print(f"Box {j+1}:", [i for i in model.I if model.x[i, j].value == 1])

print("\nNumber of boxes used:", model.obj())

Optimal Solution:
Box 1: []
Box 2: []
Box 3: []
Box 4: []
Box 5: []
Box 6: []
Box 7: []
Box 8: []
Box 9: []
Box 10: []
Box 11: []
Box 12: []
Box 13: []

Number of boxes used: 11.096061999080616


In [439]:
#Making the data for the problem
def BinPackingExample():
    B = 9
    w = [2,3,4,5,6,7,8]
    q = [4,2,6,6,2,2,2]
    s=[]
    for j in range(len(w)):
        for i in range(q[j]):
            s.append(w[j])
    return s,B

#print(BinPackingExample())
def FFD(s, B):
    remain = [B]
    sol = [[]]
    for item in sorted(s, reverse=True):
        for j,free in enumerate(remain):
            if free >= item:
                remain[j] -= item
                sol[j].append(item)
                break
        else:
            sol.append([item])
            remain.append(B-item)
    return sol
s,B = BinPackingExample()
n = len(s)
U = len(FFD(s, B))
print(s) #Weights
print(B) #Max bin capacity
print(n) #Number of boxes available
print(U) #Max number of boxes

[2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 7, 7, 8, 8]
9
24
13


## 5. Correct the code to verify model viability (optional)

In [449]:
model = ConcreteModel()
# Variables
model.x = Var(range(n), range(U), within=Binary, initialize=0)
model.y = Var(range(U), within=Binary, initialize=0)

    # Constraints
    # Each item must be in exactly one bin
def assignment_constraint(model, i):
    return sum(model.x[i, j] for j in range(U)) == 1
model.assignment_constraint = Constraint(range(n), rule=assignment_constraint)

    # The total weight in each bin cannot exceed the bin capacity times the bin use variable
def capacity_constraint(model, j):
    return sum(s[i] * model.x[i, j] for i in range(n)) <= B * model.y[j]
model.capacity_constraint = Constraint(range(U), rule=capacity_constraint)

    # x_ij can only be 1 if y_j is 1 (i.e., if bin j is used)
def linking_constraint(model, i, j):
    return model.x[i, j] <= model.y[j]
model.linking_constraint = Constraint(range(n), range(U), rule=linking_constraint)

    # Objective
model.obj = Objective(expr=sum(model.y[j] for j in range(U)), sense=minimize)

solver = SolverFactory('glpk')
solver.solve(model)

print("Objective Value:", model.obj())

Objective Value: 13.0


## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [452]:
print(response.text)

Let's think step by step to formulate a mathematical optimization model for the problem:

**Sets:**

*  $I$: Set of items, indexed by $i$.
*  $B$: Set of boxes, indexed by $j$.

**Parameters:**

*  $w_i$: Weight of item $i$, $\forall i \in I$.
*  $C$: Capacity of each box.
*  $N$: Maximum number of boxes allowed.

**Decision Variables:**

*  $x_{ij}$: Binary variable equal to 1 if item $i$ is packed in box $j$, 0 otherwise, $\forall i \in I, \forall j \in B$.

**Objective Function:**
Minimize the number of boxes used. 

$$\text{Minimize} \sum_{j \in B} \left( 1 - \prod_{i \in I} (1 - x_{ij}) \right) $$

This objective function utilizes a clever trick:

*   For each box $j$, if no item is packed in it (all $x_{ij} = 0$), the product term $\prod_{i \in I} (1 - x_{ij})$ evaluates to 1. Hence, $1 - \prod_{i \in I} (1 - x_{ij})$ becomes 0, indicating the box is unused.
*   If at least one item is packed in box $j$ (at least one $x_{ij} = 1$), the product term $\prod_{i \in I} (1 - x_{ij})$ 

In [453]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (You can replace this with your actual data)
items = ['A', 'B', 'C', 'D', 'E']
weights = {'A': 4, 'B': 2, 'C': 7, 'D': 5, 'E': 3}
capacity = 10
max_boxes = 5

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.B = pyo.Set(initialize=range(max_boxes))

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Variables
model.x = pyo.Var(model.I, model.B, domain=pyo.Binary)

# Objective function: Minimize the number of boxes used
def obj_rule(model):
    return sum(1 - pyo.prod(1 - model.x[i, j] for i in model.I) for j in model.B)
model.obj = pyo.Objective(rule=obj_rule)

# Constraints
# Capacity constraint
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C
model.capacity_constraint = pyo.Constraint(model.B, rule=capacity_rule)

# One box per item constraint
def on